In [4]:
## Setup
import atlite
import logging
logging.basicConfig(level=logging.INFO)

In [7]:
## Download - this works with ERA5
DS = atlite.Dataset(module="era5",
					years=slice(2010, 2010),
					months=slice(12,12))

INFO:atlite.dataset:Directory /Users/williamhonaker/desktop/davidson/data_for_geodata/era5 found, checking for completeness.
INFO:atlite.dataset:No file matching `/Users/williamhonaker/desktop/davidson/data_for_geodata/era5/2010/12/*.nc` was found!
INFO:atlite.dataset:0 files not completed.


In [8]:
if DS.prepared == False:
	DS.get_data()
    
## doesn't work because it's not coded to work
## download externally?

AttributeError: module 'atlite.datasets.era5' has no attribute 'wind_files'

In [11]:
## example from atlite docs - this works?

cutout = atlite.Cutout(name="western-europe-2011-01",
                       module="era5",
                       xs=slice(-13.6913, 1.7712),
                       ys=slice(60.8479, 49.9096),
                       years=slice(2010, 2010),
					   months=slice(12,12))

## Resulting download gives orography file, not era5 download


INFO:atlite.cutout:Cutout (western-europe-2011-01, /Users/williamhonaker/desktop/davidson/data_for_geodata/cutouts) not found or incomplete.
INFO:cdsapi:Welcome to the CDS
INFO:cdsapi:Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
INFO:cdsapi:Request is queued
INFO:cdsapi:Request is running
INFO:cdsapi:Request is completed
INFO:atlite.datasets.era5:Downloading request for 9 variables to /Users/williamhonaker/desktop/davidson/data_for_geodata/era5
INFO:cdsapi:Downloading http://136.156.132.210/cache-compute-0005/cache/data4/adaptor.mars.internal-1586470954.5244946-20362-37-9ae8ac49-0daa-44d5-a1af-c40b21c89f74.nc to /Users/williamhonaker/desktop/davidson/data_for_geodata/era5/2010/12/orography.nc (129.5K)
INFO:cdsapi:Download rate 160.2K/s                


In [12]:
## Example from Davidson's cutout_test.ipynb
## This works
cutout = atlite.Cutout(name="era5-europe-test-2011-01",
                       module="era5",
                       xs=slice(30, 41.56244222),
                        ys=slice(33.56459975, 35),
                       years=slice(2011, 2011),
                       months=slice(1,1,1) )
cutout.prepare(overwrite=True);

INFO:atlite.cutout:Cutout (era5-europe-test-2011-01, /Users/williamhonaker/desktop/davidson/data_for_geodata/cutouts) not found or incomplete.
INFO:atlite.preparation:Starting preparation of cutout 'era5-europe-test-2011-01'
INFO:atlite.preparation:1 tasks have been collected. Starting running them on all processors.
INFO:cdsapi:Welcome to the CDS
INFO:cdsapi:Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
INFO:cdsapi:Request is queued
INFO:cdsapi:Request is running
INFO:cdsapi:Request is completed
INFO:atlite.datasets.era5:Downloading request for 10 variables to /Users/williamhonaker/desktop/davidson/data_for_geodata/era5
INFO:cdsapi:Downloading http://136.156.132.110/cache-compute-0001/cache/data6/adaptor.mars.internal-1586471218.9733949-11489-3-c3630c58-d34f-478b-825e-465e82e160ff.nc to /Users/williamhonaker/desktop/davidson/data_for_geodata/era5/2011/01/112rsssstt.nc (3.3M)
INFO:cdsapi:Download rate 1.3M/s                    
INFO

In [13]:
cutout = atlite.Cutout(name="era5-europe-test-2011-01",
                       module="era5",
                       xs=slice(30, 41.56244222),
                        ys=slice(33.56459975, 35),
                       years=slice(2011, 2011),
                       months=slice(1,1,1) )
cutout.prepare();  ## prepare also works better here

INFO:atlite.cutout:All cutout (era5-europe-test-2011-01, /Users/williamhonaker/desktop/davidson/data_for_geodata/cutouts) files available.
INFO:atlite.preparation:[4, 1, 1, 10]
INFO:atlite.cutout:Cutout subset prepared: <Cutout era5-europe-test-2011-01 x=30.00-41.50 y=34.81-33.81 time=2011/1-2011/1 prepared>
INFO:atlite.preparation:The cutout is already prepared. If you want to recalculate it, supply an `overwrite=True` argument.


In [14]:
cutout

<Cutout era5-europe-test-2011-01 x=30.00-41.50 y=34.81-33.81 time=2011/1-2011/1 prepared>

In [15]:
cutout.name

'era5-europe-test-2011-01'

In [16]:
cutout.coords

Coordinates:
  * x           (x) float32 30.0 30.25 30.5 30.75 31.0 ... 40.75 41.0 41.25 41.5
  * y           (y) float32 34.815 34.565 34.315 34.065 33.815
  * time        (time) datetime64[ns] 2011-01-01 ... 2011-01-31T23:00:00
    lon         (x) float32 ...
    lat         (y) float32 ...
  * year-month  (year-month) MultiIndex
  - year        (year-month) int64 2011
  - month       (year-month) int64 1

In [17]:
cutout.meta

<xarray.Dataset>
Dimensions:     (time: 744, x: 47, y: 5, year-month: 1)
Coordinates:
  * x           (x) float32 30.0 30.25 30.5 30.75 31.0 ... 40.75 41.0 41.25 41.5
  * y           (y) float32 34.815 34.565 34.315 34.065 33.815
  * time        (time) datetime64[ns] 2011-01-01 ... 2011-01-31T23:00:00
    lon         (x) float32 ...
    lat         (y) float32 ...
  * year-month  (year-month) MultiIndex
  - year        (year-month) int64 2011
  - month       (year-month) int64 1
Data variables:
    height      (y, x) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2020-03-14 04:29:35 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...
    module:       era5
    view:         {'x': slice(30, 41.56244222, None), 'y': slice(35, 33.56459...

In [18]:
cutout.dataset_module.weather_data_config

{'_': {'tasks_func': <function atlite.datasets.era5.tasks_monthly_era5(xs, ys, yearmonths, prepare_func, **meta_attrs)>,
  'prepare_func': <function atlite.datasets.era5.prepare_month_era5(year, month, xs, ys)>,
  'template': '/Users/williamhonaker/desktop/davidson/data_for_geodata/era5/{year}/{month:0>2}/*.nc'}}

In [20]:
atlite.convert.wind(
                cutout, 
                turbine='Suzlon_S82_1.5_MW', 
                smooth=True)

<xarray.DataArray (time: 744, y: 5, x: 47)>
array([[[0.17585059, 0.12728752, 0.09389144, ..., 0.24482486,
         0.26590136, 0.33127548],
        [0.30526441, 0.24537273, 0.20106372, ..., 0.23692962,
         0.27552846, 0.34653954],
        [0.41403437, 0.35566666, 0.30941889, ..., 0.23622794,
         0.29509634, 0.38141238],
        [0.50060126, 0.44811285, 0.40082768, ..., 0.22779657,
         0.30343363, 0.38819561],
        [0.54630823, 0.51668171, 0.46812415, ..., 0.21178958,
         0.28978276, 0.36302398]],

       [[0.14542355, 0.10468268, 0.08052383, ..., 0.22082448,
         0.23688953, 0.29505605],
        [0.26840282, 0.21430758, 0.17902627, ..., 0.20817013,
         0.24003581, 0.30340013],
        [0.37895972, 0.32394826, 0.28496989, ..., 0.19868009,
         0.25007875, 0.32837512],
        [0.47018548, 0.41980596, 0.37791786, ..., 0.18034436,
         0.24832079, 0.32514499],
        [0.52096036, 0.49231552, 0.44882488, ..., 0.1576507 ,
         0.2260276 , 0.29167017]],

       [[0.11303287, 0.08259969, 0.06494006, ..., 0.19508095,
         0.20206597, 0.24906542],
        [0.22556004, 0.1825865 , 0.15490926, ..., 0.17223964,
         0.19809564, 0.24786147],
        [0.3337917 , 0.28841636, 0.25690962, ..., 0.1524236 ,
         0.19633117, 0.26164741],
        [0.42638369, 0.38369095, 0.35027227, ..., 0.12649588,
         0.18093949, 0.24647637],
        [0.48363031, 0.45992896, 0.42403598, ..., 0.10029629,
         0.1526822 , 0.20590053]],

       ...,

       [[0.02359585, 0.00749744, 0.00757594, ..., 0.32419815,
         0.31534031, 0.31654971],
        [0.06424044, 0.03129753, 0.01735956, ..., 0.28680435,
         0.27857417, 0.30480448],
        [0.10190834, 0.06999299, 0.04503626, ..., 0.2354698 ,
         0.2632154 , 0.32056992],
        [0.1322814 , 0.10812178, 0.08596768, ..., 0.23473362,
         0.30078434, 0.36345215],
        [0.15230861, 0.13800691, 0.11655071, ..., 0.25209874,
         0.32301808, 0.37924919]],

       [[0.01464503, 0.00420669, 0.00488318, ..., 0.37703854,
         0.38554025, 0.40866632],
        [0.04403847, 0.02126327, 0.01258405, ..., 0.36834105,
         0.33921684, 0.37103755],
        [0.07576762, 0.05317476, 0.03619799, ..., 0.33489621,
         0.31206435, 0.3714142 ],
        [0.10630516, 0.08982289, 0.07362712, ..., 0.32031307,
         0.3361389 , 0.41636851],
        [0.13211109, 0.12082306, 0.10473684, ..., 0.31622916,
         0.35650302, 0.43617557]],

       [[0.00881694, 0.00237817, 0.00364232, ..., 0.4220689 ,
         0.43556162, 0.45513073],
        [0.02943254, 0.01288095, 0.0077384 , ..., 0.43849979,
         0.42741226, 0.43792717],
        [0.05502293, 0.03581365, 0.02387689, ..., 0.45230827,
         0.42149979, 0.40605863],
        [0.08240749, 0.0650664 , 0.05151359, ..., 0.43262812,
         0.40673053, 0.39339172],
        [0.10615684, 0.09307515, 0.07774159, ..., 0.38963141,
         0.37986251, 0.38324568]]])
Coordinates:
    lon      (x) float32 30.0 30.25 30.5 30.75 31.0 ... 40.75 41.0 41.25 41.5
    lat      (y) float32 34.815 34.565 34.315 34.065 33.815
  * x        (x) float32 30.0 30.25 30.5 30.75 31.0 ... 40.75 41.0 41.25 41.5
  * y        (y) float32 34.815 34.565 34.315 34.065 33.815
  * time     (time) datetime64[ns] 2011-01-01 ... 2011-01-31T23:00:00

In [22]:
atlite.convert.windspd(
                cutout, 
                turbine='Vestas_V66_1750kW')

<xarray.DataArray 'wnd67m' (time: 744, y: 5, x: 47)>
array([[[6.38083333, 5.76112356, 5.23372553, ..., 7.03635301,
         7.24216777, 7.80579687],
        [7.64094793, 7.09949822, 6.65454437, ..., 6.95008024,
         7.32377793, 7.92718955],
        [8.52166917, 8.0601984 , 7.677231  , ..., 6.94617902,
         7.49226765, 8.20977502],
        [9.18063643, 8.78436359, 8.41938826, ..., 6.87516448,
         7.568201  , 8.26395741],
        [9.52568863, 9.30226614, 8.93571965, ..., 6.72020227,
         7.45113803, 8.06278476]],

       [[6.00353282, 5.41997128, 4.9862626 , ..., 6.80716878,
         6.96495388, 7.49700364],
        [7.31901827, 6.79794637, 6.41450152, ..., 6.66575983,
         6.99245356, 7.56770264],
        [8.24866608, 7.80252229, 7.46370638, ..., 6.56604219,
         7.08736359, 7.78201159],
        [8.95060401, 8.56563913, 8.24020293, ..., 6.37692775,
         7.07403028, 7.75745292],
        [9.33394571, 9.117972  , 8.78974907, ..., 6.1107713 ,
         6.8621878 , 7.46849313]],

       [[5.54505122, 5.02469115, 4.65180732, ..., 6.53480135,
         6.61441212, 7.08248996],
        [6.90797227, 6.45246813, 6.12263242, ..., 6.27823525,
         6.56908801, 7.0694472 ],
        [7.88176099, 7.49304761, 7.20976382, ..., 6.0337273 ,
         6.54908303, 7.20237108],
        [8.61589672, 8.2859657 , 8.01563452, ..., 5.70094302,
         6.38591757, 7.05875053],
        [9.05161862, 8.87303467, 8.59839629, ..., 5.31017426,
         6.04869081, 6.65773514]],

       ...,

       [[3.30043866, 2.03849571, 2.04782541, ..., 7.74253914,
         7.67157803, 7.68335601],
        [4.63289774, 3.64357891, 2.94873676, ..., 7.407948  ,
         7.35017043, 7.57987583],
        [5.37408997, 4.76676894, 4.12465923, ..., 6.93894778,
         7.21282299, 7.71782323],
        [5.83118179, 5.46732799, 5.08324585, ..., 6.94142134,
         7.54523573, 8.06647255],
        [6.08566572, 5.90534535, 5.59443183, ..., 7.11100775,
         7.7390435 , 8.19366799]],

       [[2.74180752, 1.4879675 , 1.61083384, ..., 8.16839663,
         8.24097458, 8.42020065],
        [4.09435897, 3.16819233, 2.57558605, ..., 8.08564356,
         7.86564036, 8.12474125],
        [4.89456181, 4.37136193, 3.84265127, ..., 7.81966181,
         7.63928243, 8.12913632],
        [5.43950503, 5.15412216, 4.8511892 , ..., 7.71142743,
         7.84353444, 8.48063578],
        [5.82818393, 5.65840991, 5.41628494, ..., 7.68006811,
         8.00911919, 8.63272189]],

       [[2.19496172, 0.97767519, 1.3409592 , ..., 8.51647457,
         8.62557038, 8.77577121],
        [3.5672532 , 2.59914666, 2.06648707, ..., 8.6292181 ,
         8.5597746 , 8.64333539],
        [4.41676314, 3.826619  , 3.3149997 , ..., 8.7374332 ,
         8.51311089, 8.39972491],
        [5.01656546, 4.65089233, 4.32052244, ..., 8.59938294,
         8.40371865, 8.30392083],
        [5.43686759, 5.2138668 , 4.93065741, ..., 8.27327556,
         8.19752712, 8.22564892]]])
Coordinates:
    lon      (x) float32 30.0 30.25 30.5 30.75 31.0 ... 40.75 41.0 41.25 41.5
    lat      (y) float32 34.815 34.565 34.315 34.065 33.815
  * x        (x) float32 30.0 30.25 30.5 30.75 31.0 ... 40.75 41.0 41.25 41.5
  * y        (y) float32 34.815 34.565 34.315 34.065 33.815
  * time     (time) datetime64[ns] 2011-01-01 ... 2011-01-31T23:00:00
Attributes:
    long_name:  extrapolated 67 m wind speed using log ratio, from nearest he...
    units:      m s**-1

In [33]:
## doesn't work, investigate parameters
atlite.convert.windwpd(
                cutout, 
                turbine='Vestas_V66_1750kW')

KeyError: 'rhoa'

In [44]:
## solar variables
## pv
ds = atlite.convert.pv(cutout, panel="KANENA", orientation = "latitude_optimal")

In [45]:
ds

<xarray.DataArray 'AC power' (y: 5, time: 744, x: 47)>
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])
Coordinates:
    lon      (x) float32 30.0 30.25 30.5 30.75 31.0 ... 40.75 41.0 41.25 41.5
    lat      (y) float32 34.815 34.565 34.315 34.065 33.815
  * x        (x) float32 30.0 30.25 30.5 30.75 31.0 ... 40.75 41.0 41.25 41.5
  * y        (y) float32 34.815 34.565 34.315 34.065 33.815
  * time     (time) datetime64[ns] 2011-01-01 ... 2011-01-31T23:00:00

In [46]:
df =ds.to_dataframe(name='pv')

In [47]:
df

lon        lat   pv
y         time                x                           
34.814999 2011-01-01 00:00:00 30.00  30.00  34.814999  0.0
                              30.25  30.25  34.814999  0.0
                              30.50  30.50  34.814999  0.0
                              30.75  30.75  34.814999  0.0
                              31.00  31.00  34.814999  0.0
...                                    ...        ...  ...
33.814999 2011-01-31 23:00:00 40.50  40.50  33.814999  0.0
                              40.75  40.75  33.814999  0.0
                              41.00  41.00  33.814999  0.0
                              41.25  41.25  33.814999  0.0
                              41.50  41.50  33.814999  0.0

[174840 rows x 3 columns]